<a href="https://colab.research.google.com/github/dtabuena/EphysLib/blob/main/Fast_Live_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'Get Standard Modules'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from scipy.signal import butter,filtfilt
from tqdm import tqdm
import warnings
from IPython.display import clear_output
from datetime import datetime
import sys
import warnings
import shutil
from google.colab import files
warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
clear_output(wait=False)



!pip install openpyxl
!pip install XlsxWriter


'''Get Repositories'''
try: shutil.rmtree('/content/EphysLib')
except: None

"run dtabuena's ephys notebooks"
!git clone https://github.com/dtabuena/EphysLib
to_import = [
            'ABF_Quality_Control.ipynb',
            'Basic_Ephys.ipynb',
            'Simple_ABF_tools.ipynb',
            'fun_math.ipynb',
            'importing_abfs_from_dropbox.ipynb',
            'QC_recoding_dataframe.ipynb',
            'Analyzers/input_resistance_analyzer.ipynb',
            'Analyzers/gain_analyzer.ipynb',
            'Analyzers/latencey_analyzer.ipynb',
            'Analyzers/IV_analyzer.ipynb',
            'Analyzers/Vm_analyzer.ipynb',
            'Analyzers/membrane_analyzer.ipynb',
            'Analyzers/rheobase_analyzer.ipynb',
            'Ephys_wrapper.ipynb',
            ]
for i in to_import:
    f = '/content/EphysLib/' + i
    %run $f


# try: shutil.rmtree('/content/CrispyMice')
# except: None
# !git clone https://github.com/dtabuena/CrispyMice
# to_import = ['CrispyPipelineFuncs.ipynb']
# for i in to_import:
#     f = '/content/CrispyMice/' + i
#     %run $f


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 3.0 MB/s eta 0:00:00
Cloning into 'EphysLib'...
remote: Enumerating objects: 813, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 813 (delta 160), reused 108 (delta 108), pack-reused 623
Receiving objects: 100% (813/813), 14.28 MiB | 14.91 MiB/s, done.
Resolving deltas: 100% (537/537), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 KB 4.7 MB/s eta 0:00:00


In [2]:
##### Setup Protocol List
VC_prot = ['VC - MemTest-10ms-160ms',
           'VC - Multi IV - 150ms',]
IC_prot = ['IC - Gain - D20pA',
           'IC - Gain - D50pA',
           'IC - Rheobase',
           'IC - R input',
           'IC - Latentcy 800pA-1s'
           'VC - 3min GapFree',
           'I0 - 3min GapFree']

In [ ]:
dataset = {'data_name': 'Fast_Data',
           'data_source': "https://www.dropbox.com/sh/s3hvu2hl27z35x8/AABrX4WaMXgDyi2kXLCKtWqAa?dl=0",
           'file_naming_scheme': ['Rec_date','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type'],
           }

Fast_results = ephys_wrapper(dataset,VC_prot,IC_prot,strat_cols=[ 'Cell_Type'])

In [ ]:
Fast_results.keys()
Fast_results['protocol_set']

['IC - Latentcy 800pA-1s',
 'None',
 'VC - InAct',
 'IC - Rheobase',
 'IC - Gain - D20pA',
 'I0 - 3min GapFree',
 'VC - Multi IV - 150ms',
 'VC - MemTest-10ms-160ms',
 'IC - R input']

In [15]:
reorg_abfs(new_dir='reorg', look_up_name='/content/Fast_Data_Recording_LookUp.csv',data_dir = '/content/my_ephys_data' )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
single_val_strat_groups = ['ap_amplitutude',
                            'Rmp_mV',
                            'Ra_160.0',
                            'Rm_160.0',
                            'Cm_pc_10.0',                    
                            'Cmq_160.0',
                            'Ra_160.0',
                            'Rm_160.0',
                            'Gain_(HzpA)',
                            'Gain_(%pA)',
                            'Rheobase',
                            'AP_thresh_US',
                            'fast_after_hyperpol',
                            'Spike_latency_(ms)',
                            'Input_Resistance_MO']


manual_exclusions = [''] # Dramatic shift in seal at unknown time during recording.
exclusion_overide = [''] # Vm dramatically improved prior to gain recording


strat_df_dict = Fast_results['strat_df_dict'].copy()
file_naming_scheme = ['Rec_date', 'GenoType', 'Sex', 'Age', 'Slice_Num', 'Cell_num', 'Cell_Type']
filtered_dict, fail_dict = final_qc(strat_df_dict,file_naming_scheme,qc_Rmp=-45,qc_AP_amp=40,qc_RR=.35,qc_Ra=65,manual_exclusions=manual_exclusions,exclusion_overide=exclusion_overide)
alt_strat_dict = restratify_results(filtered_dict,file_naming_scheme,single_val_strat_groups)
resp_curve_list = ['IV_Early_(V_stim)','IV_Early_(I_peak)','IV_Steady_State_(I_mean)','Stim_Levels_(pA)','Spike_Counts']
response_curve_data = stratify_response_curve(filtered_dict,resp_curve_list,strat_list=[''])
alt_strat_dict.update(response_curve_data)

write_strat_dfs(alt_strat_dict, dataset['data_name']+'_results_stratified_alternate')